In [1]:
!pip install google-cloud-aiplatform
!pip install s3fs
!pip install boto3

In [1]:
import base64
from io import BytesIO
import mimetypes
import os
import tempfile
import time
from urllib.parse import urlparse
import pandas as pd
import requests
import vertexai
# from vertexai.preview.generative_models import GenerativeModel, Image, Part

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

import numpy
import pandas as pd
import json


In [35]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [13]:

project_id = "abg-intelact-genai-platform"
vertexai.init(project=project_id)


In [4]:

def set_gcloud_adc_env():
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "abg-intelact-genai-platform-0c51beb9d67c.json"
    print("Successfully set environment credentials.")


In [6]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )
    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

In [13]:
def get_gemini_multiPrompt_response(image, _prompt):
    contents = [image, _prompt]
    responses = multimodal_model.generate_content(contents, stream=True)
    _sentence = ""
    for r in responses:
        _sentence = _sentence+ " " + str(r.text)
    return _sentence


In [ ]:
PROMT = [

    """
    Brand and Heritage: 
    • Brand: Identify the brand (if recognizable). 
    • Collaborations: Highlight any known heritage, including significant collaborations (e.g., with designers or other brands) or endorsements (celebrity or influencer affiliations) tied to the product. 
    • Limited Edition: Mention if the product is part of any special collections or limited editions.

    Product Overview: 
    • Category: Determine the product category (e.g., apparel, footwear, accessories). 
    • Sub-category: Specify the sub-category within the category (e.g., sneakers, handbags, jackets). 
    • Type: Define the exact product type (e.g., high-top sneakers, leather backpack, bomber jacket).

    Design & Aesthetics: 
    • Dominant colour: Describe the product’s dominant color
    • Secondary colour: Describe any secondary colors
    • Design Elements: Highlight key design elements, such as patterns, materials, textures, or unique features (e.g., embellishments, stitching, logos). 
    • Aesthetic Type: Determine the product’s overall aesthetic type (e.g., minimalistic, bold, vintage, modern, avant-garde, classic, casual), and assess how this influences the product's visual appeal and target audience. 
    • Silhouette: Examine the overall shape or silhouette of the product, identifying whether it is structured, relaxed, form-fitting, oversized, or any other distinct style. 
    • Branding Style: Examine the type of branding on the product, specifying whether it is bold (e.g., prominent logos, distinctive brand marks), subtle (e.g., discreet logos, minimal brand markings), or any other notable style that adds to the product's visual identity. 
    • Product Story: Identify and describe any narrative or concept that is conveyed through the design elements. Discuss how the design reflects a particular theme, cultural reference, or lifestyle message and how it connects to the brand's identity or story.


    • Pricing: Based on the brand, product type, and potential collaborations or endorsements, suggest a tentative price range in USD. 
    • Market Positioning: Evaluate the product’s market positioning (e.g., luxury, premium, mass-market, niche). 
    • Willingness to Pay: Estimate the target consumer's willingness to pay for the product.

    • Perceived Value Score: Based on brand prestige, exclusivity, design, and endorsements (scale of 1-10).

    • Price Elasticity: Low or high based on how much the consumer is willing to pay above or below average for similar products in the market.

    Price Ranges: 
    • Suggest a price range and the corresponding consumer group most likely to pay that amount (e.g., $200-$300 for mass-market consumers, $400+ for premium consumers).

    Desirability Index: 
    • Quantify the overall desirability of the product (scale of 1-10) based on trends, market positioning, and the product story.

    Target Consumer Profile: 
    • Demographics: Suggest the likely age group, gender, and socioeconomic background of the target consumer. 
    • Lifestyle: Describe the typical lifestyle preferences of the consumer (e.g., urban, rural, professional, creative, athletic). 
    • Fashion Style: Define the consumer’s fashion sense (e.g., casual, formal, trendy, avant-garde, luxury).

    Occasions for Use: 
    • Suggest specific occasions or contexts where this product could be worn or used by the target consumer: 
    • Social events: (e.g., parties, weddings, or dinners). 
    • Professional settings: (e.g., work meetings, corporate events). 
    • Casual outings: (e.g., coffee dates, shopping trips). 
    • Seasonal use: (e.g., summer vacations, winter getaways). 
    • Sports or activities: (e.g., gym, hiking, outdoor events). 
    • Special or cultural occasions: (e.g., festivals, religious ceremonies). 
    • Travel or leisure: (e.g., holidays, weekend trips, casual city strolls).

    Heritage and Exclusivity: 
    • Assess whether the product has any historical significance or represents part of a heritage collection. 
    • Identify if the product is associated with exclusivity through limited editions, collaborations, or rare releases that might elevate its appeal to collectors or fashion enthusiasts.
 
    Provide all the above information in json format.
    
    """
    
]

In [130]:
PROMT = [

    """

    • Brand: Identify the brand (if recognizable). 
    • Collaborations: Highlight any known heritage, including significant collaborations (e.g., with designers or other brands) or endorsements (celebrity or influencer affiliations) tied to the product. 
    • Limited Edition: Mention if the product is part of any special collections or limited editions.

    • Category: Determine the product category (e.g., apparel, footwear, accessories). 
    • Sub-category: Specify the sub-category within the category (e.g., sneakers, handbags, jackets). 
    • Type: Define the exact product type (e.g., high-top sneakers, leather backpack, bomber jacket).

    • Dominant colour: Describe the product’s dominant color
    • Secondary colour: Describe any secondary colors
    • Design Elements: Highlight key design elements, such as patterns, materials, textures, or unique features (e.g., embellishments, stitching, logos). 
    • Aesthetic Type: Determine the product’s overall aesthetic type (e.g., minimalistic, bold, vintage, modern, avant-garde, classic, casual), and assess how this influences the product's visual appeal and target audience. 
    • Silhouette: Examine the overall shape or silhouette of the product, identifying whether it is structured, relaxed, form-fitting, oversized, or any other distinct style. 
    • Branding Style: Examine the type of branding on the product, specifying whether it is bold (e.g., prominent logos, distinctive brand marks), subtle (e.g., discreet logos, minimal brand markings), or any other notable style that adds to the product's visual identity. 
    • Product Story: Identify and describe any narrative or concept that is conveyed through the design elements. Discuss how the design reflects a particular theme, cultural reference, or lifestyle message and how it connects to the brand's identity or story.

    • Tentative Pricing: Based on the brand, product type, and potential collaborations or endorsements, suggest a tentative price range in USD. 
    • Market Positioning: Evaluate the product’s market positioning (e.g., luxury, premium, mass-market, niche). 
    • Willingness to Pay: Estimate the target consumer's willingness to pay for the product.

    • Perceived Value Score: Based on brand prestige, exclusivity, design, and endorsements (scale of 1-10).

    • Price Elasticity: Low or high based on how much the consumer is willing to pay above or below average for similar products in the market.

    • Consumer type: Suggest the corresponding consumer group most likely to pay that amount (e.g., mass-market consumers, premium consumers).

    • Desirability Index: Quantify the overall desirability of the product (scale of 1-10) based on trends, market positioning, and the product story.

    • Target consumer age group: Suggest the likely age group of the target consumer.
    • Target consumer gender: Suggest the likely gender of the target consumer.
    • Target consumer socioeconomic background: Suggest the likely socioeconomic background of the target consumer.
    • Target consumer Lifestyle: Describe the typical lifestyle preferences of the consumer (e.g., urban, rural, professional, creative, athletic). 
    • Target consumer Fashion Style: Define the consumer’s fashion sense (e.g., casual, formal, trendy, avant-garde, luxury).

    • Occasions for Use (Social): Suggest Social events where this product could be worn or used (e.g., parties, weddings, or dinners). 
    • Occasions for Use (Professional): Suggest Professional settings where this product could be worn or used  (e.g., work meetings, corporate events). 
    • Occasions for Use (Casual): Suggest Casual outings where this product could be worn or used (e.g., coffee dates, shopping trips). 
    • Occasions for Use (Seasonal): Suggest Seasonal where this product could be worn or used (e.g., summer vacations, winter getaways). 
    • Occasions for Use (Sports): Suggest Sports or activities where this product could be worn or used (e.g., gym, hiking, outdoor events). 
    • Occasions for Use (Cultural): Suggest Special Cultural occasions where this product could be worn or used (e.g., festivals, religious ceremonies). 
    • Occasions for Use (Travel): Suggest Travel or leisure where this product could be worn or used (e.g., holidays, weekend trips, casual city strolls).

    • Heritage: Assess whether the product has any historical significance or represents part of a heritage collection. 
    • Exclusivity: Identify if the product is associated with exclusivity through limited editions or collaborations or rare releases that might elevate its appeal to collectors or fashion enthusiasts.
 
    Provide all the above information in json format.
    
    """
    
]

# Load images from Cloud Storage URI

In [110]:

# Set model parameters
generation_config = GenerationConfig(
    temperature=0.4,
    top_p=1.0,
    top_k=1,
    candidate_count=1,
    max_output_tokens=8192,
)

# Set safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}


In [124]:
SingleProductURLs = [
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_1.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_2.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_3.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_4.jpg?v=1706125363',
    'https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_5.jpg?v=1706125363'
]

In [125]:
image_file = []
for im in SingleProductURLs:
    image_file.append(Part.from_uri( im,mime_type="image/png",))

image_file

[file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_1.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_2.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_3.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_4.jpg?v=1706125363"
 },
 file_data {
   mime_type: "image/png"
   file_uri: "https://amiri.com/cdn/shop/products/33_Pre-SS24_20Men_s_20Footwear_FOOTWEAR_Boot_BANDANA_20MALIBU_PS24MFB004_Wash_20Indigo_5.jpg?v=1706125363"
 }]

In [113]:
set_gcloud_adc_env()

Successfully set environment credentials.


In [131]:

# model = GenerativeModel("gemini-1.5-flash-001")
model = GenerativeModel("gemini-1.5-pro-002")

response = model.generate_content(
    image_file
    +
    PROMT,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream = False)

model_output = response.text
model_output = model_output.replace("```","").replace("json","")
model_output_json = json.loads(model_output)
model_output_df = pd.json_normalize(model_output_json)
model_output_df

,Brand,Collaborations,Limited Edition,Category,Sub-category,Type,Dominant colour,Secondary colour,Design Elements,Aesthetic Type,Silhouette,Branding Style,Product Story,Tentative Pricing,Market Positioning,Willingness to Pay,Perceived Value Score,Price Elasticity,Consumer type,Desirability Index,Target consumer age group,Target consumer gender,Target consumer socioeconomic background,Target consumer Lifestyle,Target consumer Fashion Style,Occasions for Use (Social),Occasions for Use (Professional),Occasions for Use (Casual),Occasions for Use (Seasonal),Occasions for Use (Sports),Occasions for Use (Cultural),Occasions for Use (Travel),Heritage,Exclusivity
0,AMIRI,None,False,Footwear,Shoes,Slip-on Sneakers/Boots,Light Blue,Beige/Cream,"Paisley bandana print on denim upper, shearlin...","Casual, streetwear, luxury","Low-top, chunky","Subtle, with small logo placements",Blends casual comfort with a touch of luxury a...,$600 - $800,Luxury,$700 - $900,8,Low,"Premium consumers, fashion enthusiasts",7,25-45,Male/Unisex,Upper-middle class to affluent,"Urban, fashion-conscious, appreciates luxury a...",Casual streetwear with high-end touches.,"Casual outings with friends, weekend brunches,...","Not ideal for formal business settings, but su...","Everyday wear, running errands, coffee dates, ...",Suitable for transitional seasons like spring ...,Not suitable for sports activities.,"Music festivals, art exhibitions, cultural eve...","Comfortable for airport travel, sightseeing, a...","No significant historical significance, but dr...","Not explicitly stated as limited edition, but ..."


In [133]:
len(model_output_df.columns)

34

# Multiple Products

In [37]:
import ray
import os
import pandas as pd
import s3fs
import sys
import json
import boto3
from io import StringIO
import datetime
import time
import vertexai
from vertexai.generative_models import GenerativeModel, Image, Part

/Users/abmcpl.itsupport/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-05 12:05:42,513	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [114]:
with open('s3connection.json', 'r') as openfile:
    pwd = json.load(openfile)

In [115]:
def load_and_prepare_data(filename, pwd):

    s3 = s3fs.S3FileSystem(anon=False,key=pwd['key'],secret=pwd['secret'])
    s3bucket="oab-aw-data-insights-s3"

    try:
        df = pd.read_csv(filename)
    except:
        df = pd.read_csv(s3.open(os.path.join(filename),'rb'), encoding='Latin-1')

    print(df.Brand.value_counts().reset_index())
    _brands = list(df.Brand.unique())

    # _brands = ['Golden Goose', 'Autry']
    # df0 = df[df.Brand == _brands[0]].head(20)
    # df1 = df[df.Brand == _brands[1]].head(20)
    # df = pd.concat([df0, df1])

    # df0 = df[df.Brand == _brands[0]].head(100)
    # df1 = df[df.Brand == _brands[1]].head(100)
    # df2 = df[df.Brand == _brands[2]].head(100)
    # df3 = df[df.Brand == _brands[3]].head(100)
    # df4 = df[df.Brand == _brands[4]].head(100)
    # df = pd.concat([df0, df1, df2, df3, df4])

    df = df.reset_index()
    df.drop(columns='index', inplace=True)

    print(df.shape)
    print(df.Brand.value_counts().reset_index())

    print(df.head(3))

    return df

In [116]:
url = 'Product Image'

filename = 's3://oab-aw-data-insights-s3/datarepo/transformed/testing/brand_d2c/brandD2C_marketplace_gemini_ai_2024-10-03.csv'
# filename = 'ecommerce_marketplace_subset_gemini_ai_2024-09-29.csv'
df = load_and_prepare_data(filename, pwd)

                 index  Brand
0                  APL   1425
1  Christian Louboutin    949
2                Sorel    875
3            Represent    820
4                Amiri    734
5                Autry    641
6            Cole Haan    567
7                 Veja    560
(6571, 33)
                 index  Brand
0                  APL   1425
1  Christian Louboutin    949
2                Sorel    875
3            Represent    820
4                Amiri    734
5                Autry    641
6            Cole Haan    567
7                 Veja    560
  Country                        Website  \
0      UK  https://www.autry-usa.com/en/   
1      UK  https://www.autry-usa.com/en/   
2      UK  https://www.autry-usa.com/en/   

                                         Product URL   Product Code  \
0  https://www.autry-usa.com/en/woman/apparel/bot...  8056713266648   
1  https://www.autry-usa.com/en/woman/apparel/top...  8056713265498   
2  https://www.autry-usa.com/en/man/apparel/top/a...  80567

In [117]:
df.head(3)

,Country,Website,Product URL,Product Code,Product Name,Product Category,Product sub Category,Product type,Product Image,Brand,Colorway,Availability,Gender,Price Currency,MRP,Price,Discount,Size,Release Date,Tags,Description,attributes,Material,Country of origin,Ratings,Reviews,Vendor,Images,rank,Crawl_Date,data_name,Release_Month,datestored
0,UK,https://www.autry-usa.com/en/,https://www.autry-usa.com/en/woman/apparel/bot...,8056713266648,ACTION PANTS IN MELANGE GRAY COTTON,APPAREL,BOTTOM,NaN,https://www.autry-usa.com/upload/articoli/dett...,Autry,GREY,InStock,WOMAN,EUR,160.0,160.0,0.0,NaN,NaN,NaN,Stretch waist cotton jersey pantsSPECIFICATION...,"['Drawstring at waist.', 'Side pocket.', 'Cont...",Composition: 100% COTTON,NaN,NaN,NaN,AUTRY,['https://www.autry-usa.com/upload/articoli/zo...,1,20240927,autry_uk_report_09282024.csv,NaN,2024-10-05
1,UK,https://www.autry-usa.com/en/,https://www.autry-usa.com/en/woman/apparel/top...,8056713265498,ANORAK JACKET IN AZURE NYLON,APPAREL,TOP,NaN,https://www.autry-usa.com/upload/articoli/dett...,Autry,SKY BLUE,InStock,WOMAN,EUR,190.0,190.0,0.0,NaN,NaN,NaN,Tech fabric Anorak.SPECIFICATIONS:Color matche...,"['Color matched hood.', 'Sleeves with elastica...",Composition: 100% NYLON,NaN,NaN,NaN,AUTRY,['https://www.autry-usa.com/upload/articoli/zo...,2,20240927,autry_uk_report_09282024.csv,NaN,2024-10-05
2,UK,https://www.autry-usa.com/en/,https://www.autry-usa.com/en/man/apparel/top/a...,8056713268048,ANORAK JACKET IN AZURE NYLON,APPAREL,TOP,NaN,https://www.autry-usa.com/upload/articoli/dett...,Autry,WHITE/SKY BLUE,InStock,MAN,EUR,210.0,210.0,0.0,NaN,NaN,NaN,Two-tone tech fabric Anorak.SPECIFICATIONS:Col...,['Color matched hood with adjustable stretch d...,Composition: 100% NYLON,NaN,NaN,NaN,AUTRY,['https://www.autry-usa.com/upload/articoli/zo...,3,20240927,autry_uk_report_09282024.csv,NaN,2024-10-05


In [118]:
df_brandD2C = df[(df.Brand!='Autry') & 
                 (df['Product Category']=='Footwear')
                ][['Brand', 'Product URL', 'Product Image', 'Images']]

In [119]:
PRODUCT_URL = list(df_brandD2C['Product Image'].unique()[:3]) + list(df_brandD2C['Product Image'].unique()[-3:])

In [120]:
PRODUCT_URL

['https://amiri.com/cdn/shop/files/d7dc9e74-a124-4545-b9d2-b6ec2076565c_large.jpg?v=1723510090',
 'https://amiri.com/cdn/shop/products/6_Pre-SS24_FOOTWEAR_Sneaker_STARS_20LOW_PS24WFS004_Birch_1_0cfea535-e7c0-44f6-a992-e09ab20b40b5_large.jpg?v=1705778127',
 'https://amiri.com/cdn/shop/products/9_Pre-SS24_FOOTWEAR_Sneaker_MA_20RUNNER_PS24WFS019_Brown_1_large.jpg?v=1706914259',
 'https://columbia.scene7.com/is/image/ColumbiaSportswear2/2058681_309_f?wid=768&hei=806&v=1724853763',
 'https://columbia.scene7.com/is/image/ColumbiaSportswear2/2058681_862_f?wid=768&hei=806&v=1724853763',
 'https://columbia.scene7.com/is/image/ColumbiaSportswear2/2069871_617_f?wid=768&hei=806&v=1724853763']

In [134]:
import ast

model_output_combined_df = pd.DataFrame()

for U in PRODUCT_URL:
    
    try:
        print(U)
        SingleProductURLs = df_brandD2C[df_brandD2C['Product Image'] == U]['Images'].unique()[0]
        SingleProductURLs = ast.literal_eval(SingleProductURLs)

        image_file = []
        for im in SingleProductURLs[:4]:
            image_file.append(Part.from_uri( im,mime_type="image/png",))


        model = GenerativeModel("gemini-1.5-pro-002")

        response = model.generate_content(
            image_file
            +
            PROMT,
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream = False)

        model_output = response.text
        model_output = model_output.replace("```","").replace("json","")
        model_output_json = json.loads(model_output)
        model_output_df = pd.json_normalize(model_output_json)
        model_output_df['Product Image'] = U

        model_output_combined_df = pd.concat([model_output_combined_df, model_output_df])
        
    except :
        print('JSONDecodeError')

https://amiri.com/cdn/shop/files/d7dc9e74-a124-4545-b9d2-b6ec2076565c_large.jpg?v=1723510090
https://amiri.com/cdn/shop/products/6_Pre-SS24_FOOTWEAR_Sneaker_STARS_20LOW_PS24WFS004_Birch_1_0cfea535-e7c0-44f6-a992-e09ab20b40b5_large.jpg?v=1705778127
https://amiri.com/cdn/shop/products/9_Pre-SS24_FOOTWEAR_Sneaker_MA_20RUNNER_PS24WFS019_Brown_1_large.jpg?v=1706914259
https://columbia.scene7.com/is/image/ColumbiaSportswear2/2058681_309_f?wid=768&hei=806&v=1724853763
https://columbia.scene7.com/is/image/ColumbiaSportswear2/2058681_862_f?wid=768&hei=806&v=1724853763
https://columbia.scene7.com/is/image/ColumbiaSportswear2/2069871_617_f?wid=768&hei=806&v=1724853763


In [135]:
model_output_combined_df = model_output_combined_df.reset_index()
model_output_combined_df.drop(columns='index', inplace=True)
model_output_combined_df

,Brand,Collaborations,Limited Edition,Category,Sub-category,Type,Dominant colour,Secondary colour,Design Elements,Aesthetic Type,Silhouette,Branding Style,Product Story,Tentative Pricing,Market Positioning,Willingness to Pay,Perceived Value Score,Price Elasticity,Consumer type,Desirability Index,Target consumer age group,Target consumer gender,Target consumer socioeconomic background,Target consumer Lifestyle,Target consumer Fashion Style,Occasions for Use (Social),Occasions for Use (Professional),Occasions for Use (Casual),Occasions for Use (Seasonal),Occasions for Use (Sports),Occasions for Use (Cultural),Occasions for Use (Travel),Heritage,Exclusivity,Product Image
0,AMIRI,None evident from the image.,Unknown,Footwear,Sneakers,Low-top sneakers,Off-white/Cream,Black,"Skeleton bone pattern on side panels, perforat...","Modern, streetwear, edgy",Classic low-top sneaker silhouette,"Subtle, with small AMIRI logos on the side and...",The skeleton bone pattern adds a rebellious an...,$600 - $800,Luxury,$600 - $800,8,Low,"Premium consumers, streetwear enthusiasts, AMI...",7,18-35,"Male (primarily), Female",Upper-middle class to affluent,"Urban, fashion-forward, interested in music an...","Streetwear, high-fashion, edgy, casual.","Casual parties, concerts, music festivals, soc...","Not suitable for formal professional settings,...","Everyday wear, shopping, errands, meeting friends","Year-round, but particularly suitable for spri...","Not specifically designed for sports, but coul...","Music festivals, art exhibitions, fashion events","City breaks, weekend getaways, casual travel",No significant historical heritage evident.,Potentially exclusive depending on release str...,https://amiri.com/cdn/shop/files/d7dc9e74-a124...
1,AMIRI,None evident,Unknown,Footwear,Sneakers,Low-top sneakers,Off-white/Cream,Light beige/Tan,"Cutout star pattern, perforated leather, AMIRI...","Modern, casual, streetwear-inspired with a tou...",Classic low-top sneaker silhouette,"Subtle, with small AMIRI branding on the heel ...",The design blends casual streetwear aesthetics...,$600 - $800,Premium/Luxury,$500 - $700,7/10,Low,"Premium consumers, fashion-conscious individua...",7/10,25-45,Male/Unisex,Upper-middle class to affluent,"Urban, fashion-forward, appreciates quality an...","Casual, streetwear, luxury casual","Casual outings with friends, social events, co...","Creative industries, casual Fridays","Everyday wear, shopping, errands, coffee dates","Spring, Summer, Fall",Not ideal for sports activities,"Music festivals, art exhibitions","City breaks, weekend getaways",No significant historical heritage,"Not explicitly stated as exclusive, but AMIRI'...",https://amiri.com/cdn/shop/products/6_Pre-SS24...
2,AMIRI,None,False,Footwear,Sneakers,Low-top sneakers,Beige/Off-white,Brown/Taupe,"Mesh panels, suede overlays, AMIRI logo on ton...","Modern, sporty, luxury streetwear","Chunky, athletic","Subtle, with logo placement on tongue and heel",The design combines athletic-inspired elements...,$700 - $900,Luxury,$700 - $900,8,Low,"Premium consumers, fashion enthusiasts, luxury...",7,25-45,Male/Female,Upper-middle class to affluent,"Urban, fashion-forward, appreciates luxury and...","Casual, streetwear, luxury casual","Casual outings, social events, parties, concerts","Creative industries, casual Fridays","Shopping, coffee dates, errands","Year-round, particularly suitable for transiti...","Not ideal for high-impact sports, but suitable...","Fashion events, art exhibitions","City breaks, weekend trips",Reflects AMIRI's established aesthetic and bra...,"While not a limited edition, the brand's luxur...",https://amiri.com/cdn/shop/products/9_Pre-SS24...
3,Sorel,None,False,Footwear,Sneakers,Mid-top puffer sneakers,Neon Green/Lime Green,"Off-white/Cream, Dark Navy/Grey","Puffy nylon upper, chunky sculpted midsole, si...","Modern, sporty, casual, trendy","Chunky, voluminous","Subtle, with small Sorel logo on the side.",Combines Sorel's heritage of functional footwe...,$

In [136]:
df_result = pd.merge(df_brandD2C, model_output_combined_df, on='Product Image', how='inner')

In [147]:
df_result.head(3)

,Brand_x,Product URL,Product Image,Images,Brand_y,Collaborations,Limited Edition,Category,Sub-category,Type,Dominant colour,Secondary colour,Design Elements,Aesthetic Type,Silhouette,Branding Style,Product Story,Tentative Pricing,Market Positioning,Willingness to Pay,Perceived Value Score,Price Elasticity,Consumer type,Desirability Index,Target consumer age group,Target consumer gender,Target consumer socioeconomic background,Target consumer Lifestyle,Target consumer Fashion Style,Occasions for Use (Social),Occasions for Use (Professional),Occasions for Use (Casual),Occasions for Use (Seasonal),Occasions for Use (Sports),Occasions for Use (Cultural),Occasions for Use (Travel),Heritage,Exclusivity
0,Amiri,https://amiri.com/en-gb/products/skel-top-low-...,https://amiri.com/cdn/shop/files/d7dc9e74-a124...,['https://amiri.com/cdn/shop/files/d7dc9e74-a1...,AMIRI,None evident from the image.,Unknown,Footwear,Sneakers,Low-top sneakers,Off-white/Cream,Black,"Skeleton bone pattern on side panels, perforat...","Modern, streetwear, edgy",Classic low-top sneaker silhouette,"Subtle, with small AMIRI logos on the side and...",The skeleton bone pattern adds a rebellious an...,$600 - $800,Luxury,$600 - $800,8,Low,"Premium consumers, streetwear enthusiasts, AMI...",7,18-35,"Male (primarily), Female",Upper-middle class to affluent,"Urban, fashion-forward, interested in music an...","Streetwear, high-fashion, edgy, casual.","Casual parties, concerts, music festivals, soc...","Not suitable for formal professional settings,...","Everyday wear, shopping, errands, meeting friends","Year-round, but particularly suitable for spri...","Not specifically designed for sports, but coul...","Music festivals, art exhibitions, fashion events","City breaks, weekend getaways, casual travel",No significant historical heritage evident.,Potentially exclusive depending on release str...
1,Amiri,https://amiri.com/en-gb/products/women-stars-l...,https://amiri.com/cdn/shop/products/6_Pre-SS24...,['https://amiri.com/cdn/shop/products/6_Pre-SS...,AMIRI,None evident,Unknown,Footwear,Sneakers,Low-top sneakers,Off-white/Cream,Light beige/Tan,"Cutout star pattern, perforated leather, AMIRI...","Modern, casual, streetwear-inspired with a tou...",Classic low-top sneaker silhouette,"Subtle, with small AMIRI branding on the heel ...",The design blends casual streetwear aesthetics...,$600 - $800,Premium/Luxury,$500 - $700,7/10,Low,"Premium consumers, fashion-conscious individua...",7/10,25-45,Male/Unisex,Upper-middle class to affluent,"Urban, fashion-forward, appreciates quality an...","Casual, streetwear, luxury casual","Casual outings with friends, social events, co...","Creative industries, casual Fridays","Everyday wear, shopping, errands, coffee dates","Spring, Summer, Fall",Not ideal for sports activities,"Music festivals, art exhibitions","City breaks, weekend getaways",No significant historical heritage,"Not explicitly stated as exclusive, but AMIRI'..."
2,Amiri,https://amiri.com/en-gb/products/women-ma-runn...,https://amiri.com/cdn/shop/products/9_Pre-SS24...,['https://amiri.com/cdn/shop/products/9_Pre-SS...,AMIRI,None,False,Footwear,Sneakers,Low-top sneakers,Beige/Off-white,Brown/Taupe,"Mesh panels, suede overlays, AMIRI logo on ton...","Modern, sporty, luxury streetwear","Chunky, athletic","Subtle, with logo placement on tongue and heel",The design combines athletic-inspired elements...,$700 - $900,Luxury,$700 - $900,8,Low,"Premium consumers, fashion enthusiasts, luxury...",7,25-45,Male/Female,Upper-middle class to affluent,"Urban, fashion-forward, appreciates luxury and...","Casual, streetwear, luxury casual","Casual outings, social events, parties, concerts","Creative industries, casual Fridays","Shopping, coffee dates, errands","Year-round, particularly suitable for transiti...","Not ideal for high-impact sports, but suitable...","Fashion events, art exhibitions","City breaks, weekend trips",Reflects AMIRI's established aesthetic and bra...,"While not a limited edi

In [146]:
df_result.iloc[:,1][2]

'https://amiri.com/en-gb/products/women-ma-runner-brown'

In [148]:
df_result.to_csv('Sample_GeminiPro_SingleLargePrompt.csv', index=False)